In [ ]:
%matplotlib inline
from __future__ import print_function
import os
import sys
import copy
import random
import traceback
import numpy as np
import tensorflow as tf
import lxml.etree as et
from six.moves import cPickle as pickle

import convnet
import mutate
import convevo
import darwin

In [ ]:
reload (convnet)
reload (mutate)
reload (convevo)
reload (darwin)

In [ ]:
# http://stackoverflow.com/questions/29772158/make-ipython-notebook-print-in-real-time
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [ ]:
evo = darwin.Darwin(range(0, 10), lambda i: str(i), lambda i, e: i, lambda ps, o, e: ps[0] + e.randint(1,20))
results = evo.evaluate(random.Random(42))
evo.repopulate(0.1, 2, results, {}, random.Random(42))
evo.evaluate(random.Random(42))
print(evo.best())

In [ ]:
start_a = 0
start_b = 100
for a in range(0,5):
    for b in range(0,5):
        for i in range(3):
            list_a = range(start_a, start_a + a)
            list_b = range(start_b, start_b + b)
            cross = mutate.cross_lists(list_a, list_b, random)
            print(len(list_a), len(list_b), len(cross), cross)

In [ ]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [ ]:
def batch_accuracy(session, graph_info, inputs, labels, batch_size):
    total_accuracy = 0
    batch_count = len(inputs) / batch_size
    for b in xrange(batch_count):
        batch_data = inputs[b * batch_size: (b + 1) * batch_size]
        predictions = session.run([graph_info["verify_predictions"]], feed_dict={graph_info["verify"] : batch_data})[0]
        total_accuracy += accuracy(predictions, labels[b * batch_size: (b + 1) * batch_size]) / float(batch_count)
    return total_accuracy

In [ ]:
def run_graph(graph_info, data, step_count, report_every=50, verbose=True, accuracy_minimum=None):
    with tf.Session(graph=graph_info["graph"]) as session:
        tf.initialize_all_variables().run()
        print("Initialized")
        batch_size = graph_info["batch_size"]
        valid_accuracy = 0
        for step in xrange(step_count + 1):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (data["train_labels"].shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = data["train"][offset:(offset + batch_size), :, :, :]
            batch_labels = data["train_labels"][offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            targets = [graph_info["optimizer"], graph_info["loss"], graph_info["predictions"]]
            feed_dict = {graph_info["train"] : batch_data, graph_info["labels"] : batch_labels}
            _, l, predictions = session.run(targets, feed_dict=feed_dict)
            if np.isnan(l):
                print("Error computing loss")
                return 0
            if (step % report_every == 0):
                if verbose:
                    print("Minibatch loss at step", step, ":", l)
                    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                valid_accuracy = batch_accuracy(session, graph_info, data["valid"], data["valid_labels"], batch_size)
                print("Validation accuracy: %.1f%%" % valid_accuracy)
                if accuracy_minimum and step > 0 and valid_accuracy < accuracy_minimum:
                    print("Early out.")
                    break
        if verbose:
            test_accuracy = batch_accuracy(session, graph_info, data["test"], data["test_labels"], batch_size)
            print("Test accuracy: %.1f%%" % test_accuracy)
        return valid_accuracy

In [ ]:
def graph_input_shape(batch_size, data):
    image_size = data["image_size"]
    channel_count = data["channel_count"]
    return (batch_size, image_size, image_size, channel_count)

def setup_evo_graph(
    batch_size,
    data,
    evo_stack
):
    graph = tf.Graph()
    with graph.as_default():
        # Input data.
        input_shape = graph_input_shape(batch_size, data)
        output_shape = (batch_size, data["label_count"])
        train = tf.placeholder(tf.float32, shape=input_shape)
        labels= tf.placeholder(tf.float32, shape=output_shape)
        verify= tf.placeholder(tf.float32, shape=input_shape)

        layers = evo_stack.construct(input_shape)
        l2_loss = 0
        
        for layer in layers:
            layer.setup_parameters()
            l2_loss = layer.update_loss(l2_loss)
        
        def model(nodes, train):
            for layer in layers:
                nodes.append(layer.connect(nodes[-1], train))
            return nodes[-1]

        logits = model([train], True)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels)) + l2_loss
        
        global_step = tf.Variable(0)
        optimizer = evo_stack.construct_optimizer(global_step)
        
        info = {
            "graph": graph,
            "batch_size": batch_size,
            "train": train,
            "labels": labels,
            "loss": loss,
            "optimizer": optimizer.minimize(loss, global_step=global_step),

            # Predictions for the training, validation, and test data.
            "predictions": tf.nn.softmax(logits),
            "verify": verify,
            "verify_predictions": tf.nn.softmax(model([verify], False))
        }
    return info

In [ ]:
def create_evo_stack(patch_size, stride, depth, hidden_size, label_count, init_scale, optimizer_name=None):
    if optimizer_name:
        optimizer = convevo.Optimizer(optimizer_name, 0.05)
        optimizer.default_parameters()
    else:
        optimizer = None
    stack = convevo.LayerStack(flatten=True, optimizer=optimizer)
    default_init = lambda: convevo.Initializer("normal", scale=init_scale)

    stack.add_layer(convevo.ImageLayer("conv_bias", patch_size, stride, depth, "SAME", default_init()), relu=True)
    stack.add_layer(convevo.ImageLayer("conv_bias", patch_size, stride, depth, "SAME", default_init()), relu=True)
    stack.add_layer(convevo.HiddenLayer(hidden_size, bias=True, initializer=default_init()), relu=True)
    stack.add_layer(convevo.HiddenLayer(label_count, bias=True, initializer=default_init()), relu=False)
    
    return stack

In [ ]:
pickle_file = '../ud730/notMNIST_full.pickle'

train_dataset = []
train_labels = []
test_dataset = []
test_labels = []

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def setup_data(inputs_train, labels_train, inputs_test, labels_test):
    data = {
        "image_size": 28,
        "label_count": 10,
        "channel_count": 1
    }
    data["total_image_size"] = data["image_size"] * data["image_size"]

    def setup_data(inputs, labels, name):
        inputs = inputs.reshape((-1, data["image_size"], data["image_size"], data["channel_count"])).astype(np.float32)
        # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
        labels = (np.arange(data["label_count"]) == labels[:,None]).astype(np.float32)
        print(name + " set", inputs.shape, labels.shape)
        return inputs, labels
    data["train"], data["train_labels"] = setup_data(inputs_train, train_labels, "Training")
    data["test"], data["test_labels"] = setup_data(inputs_test, labels_test, "Test")
    return data

full_data = setup_data(train_dataset, train_labels, test_dataset, test_labels)
print(full_data.keys())

In [ ]:
del train_dataset
del train_labels
del test_dataset
del test_labels

In [ ]:
def setup_validate(data, train_count, validate_count, seed=None):
    if seed:
        np.random.seed(seed)

    def randomize(inputs, labels):
        permutation = np.random.permutation(labels.shape[0])
        shuffled_inputs = inputs[permutation,:,:,:]
        shuffled_labels = labels[permutation,:]
        return shuffled_inputs, shuffled_labels

    train_inputs = data["train"][:]
    train_labels = data["train_labels"][:]
    cross_data = copy.copy(data)

    train_inputs, train_labels = randomize(train_inputs, train_labels)
    cross_data["train"] = train_inputs[:train_count]
    cross_data["train_labels"] = train_labels[:train_count]

    cross_data["valid"] = train_inputs[train_count:train_count + validate_count]
    cross_data["valid_labels"] = train_labels[train_count:train_count + validate_count]
    return cross_data
cross_data = setup_validate(full_data, 1000, 100)
print(cross_data["train_labels"].shape)
print(cross_data["train_labels"][0])
print(full_data["train_labels"][0])
print(cross_data["valid"].shape)
del cross_data

In [ ]:
def eval_stack(stack, entropy):
    stack.reseed(entropy)
    
    eval_steps = 100000
    batch_size = 16
    
    data = setup_validate(full_data, eval_steps, 5000)
    
    try:
        evo_graph = setup_evo_graph(batch_size, data, stack)
    except KeyboardInterrupt:
        raise
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        print(lines[-1])
        convevo.output_error(stack, lines, "temp")
        return -10
        
    try:
        return run_graph(evo_graph, data, eval_steps, report_every=eval_steps/4, verbose=False, accuracy_minimum=50)
    except KeyboardInterrupt:
        raise
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        print(lines[-1])
        convevo.output_error(stack, lines, "temp")
        return -1

In [ ]:
def test_optimizers():
    for optimizer_name in ["GradientDescent", "Adadelta", "Adagrad", "Momentum", "Adam", "RMSProp"]:
        # As of this writing "Ftrl" is not supported on the GPU
        batch_size = 16
        eval_steps = 10000
        test_stack = create_evo_stack(5, 2, 64, 128, 10, 0.1, optimizer_name)
        test_data = setup_validate(full_data, eval_steps * batch_size, 500)
        test_graph = setup_evo_graph(batch_size, test_data, test_stack)
        print (run_graph(test_graph, test_data, eval_steps, report_every=eval_steps/4, verbose=False))
test_optimizers()

In [ ]:
test_stack = create_evo_stack(5, 2, 64, 128, 10, 0.1)
print(convevo.serialize(test_stack))
eval_stack(test_stack, random.Random(42))

In [ ]:
mutate_entropy = random.Random(42)
scored_mutants = []
breed_options = {
    "input_shape": graph_input_shape(16, full_data)
}
for mutant in convevo.init_population([create_evo_stack(5, 2, 64, 64, 10, 0.1)], 20, breed_options, mutate_entropy):
    scored_mutants.append((mutant, 0.0))
    
convevo.output_results(scored_mutants, "temp", "mutants.xml", 0, 0)

In [ ]:
mutate_entropy = random.Random(42)
mutant_children = []
for _ in range(20):
    mutant_a = mutate_entropy.choice(scored_mutants)[0]
    mutant_b = mutate_entropy.choice(scored_mutants)[0]
    mutant_children.append((convevo.breed([mutant_a, mutant_b], breed_options, mutate_entropy), 0.0))
    
convevo.output_results(mutant_children, "temp", "mutant_offspring", 42, 0)

In [ ]:
mutate_seed = random.randint(1, 100000)
print("Mutate Seed:", mutate_seed)
mutate_entropy = random.Random(mutate_seed)
eval_seed = random.randint(1, 100000)
print("Eval Seed:", eval_seed)
eval_entropy = random.Random(eval_seed)

population_size = 20
generations = 10
prototype_stack = create_evo_stack(5, 2, 64, 128, 10, 0.1)
breed_options = {
    "input_shape": graph_input_shape(16, full_data)
}

conv_population = convevo.init_population([prototype_stack], population_size, breed_options, mutate_entropy)
conv_darwin = darwin.Darwin(conv_population, convevo.serialize, eval_stack, convevo.breed)

for g in range(generations):
    print("Generation", g)
    results = conv_darwin.evaluate(eval_entropy)
    convevo.output_results(results, "temp")
    conv_darwin.repopulate(0.25, 4, results, breed_options, mutate_entropy)

In [ ]:
failed = 1
print (convevo.serialize(conv_darwin.population[failed]))
eval_stack(conv_darwin.population[failed], eval_entropy)

In [ ]:
best = conv_darwin.best()
print("Best score:", best[1])
print(convevo.serialize(best[0]))

In [ ]:
results = darwin.descending_score(conv_darwin.history.values())
convevo.output_results(results, "testing", "new_run.xml", mutate_seed, eval_seed)
len(results)

In [ ]:
second_population,m_seed,e_seed = convevo.load_population("testing/second_run.xml")
len(second_population)

In [ ]:
mutate_seed = random.randint(1, 100000)
print("Mutate Seed:", mutate_seed)
mutate_entropy = random.Random(mutate_seed)
eval_seed = random.randint(1, 100000)
print("Eval Seed:", eval_seed)
eval_entropy = random.Random(eval_seed)

population_size = 20
generations = 20

conv_population = convevo.init_population(second_population[:5], population_size, mutate_entropy)
conv_darwin = darwin.Darwin(conv_population, convevo.serialize, eval_stack, convevo.breed)

for g in range(generations):
    print("Generation", g)
    results = conv_darwin.evaluate(eval_entropy)
    convevo.output_results(results, "temp")
    conv_darwin.repopulate(0.25, 4, results, mutate_entropy)

In [ ]:
results = darwin.descending_score(conv_darwin.history.values())
convevo.output_results(results, "testing", "fifth_run.xml", mutate_seed, eval_seed)
len(results)

In [ ]:
l2_population,_,eval_seed = convevo.load_population("testing/l2_test.xml")
print(len(l2_population))

l2_darwin = darwin.Darwin(l2_population, convevo.serialize, eval_stack, convevo.breed)
l2_results = l2_darwin.evaluate(random.Random(eval_seed))

In [ ]:
errors = []

for root, dirs, files in os.walk('temp'):
    for name in files:
        path = os.path.join(root, name)
        low_name = name.lower()
        if low_name.startswith("err"):
            with open (path, "r") as error_file:
                lines=error_file.readlines()
                errors.append((path, lines[-1]))

for path, error in sorted(errors, key=lambda e: e[0]):
    print(path)
    print(error)

In [ ]:
error_population,_,eval_seed = convevo.load_population("testing/error.xml")
print(len(error_population))

error_darwin = darwin.Darwin(error_population, convevo.serialize, eval_stack, convevo.breed)
error_results = error_darwin.evaluate(random.Random(eval_seed))